# Try Pylandsat

In [1]:
pip install pylandsat

  Using cached fiona-1.10.1-cp311-cp311-win_amd64.whl.metadata (58 kB)
Using cached fiona-1.10.1-cp311-cp311-win_amd64.whl (24.5 MB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
from datetime import datetime

from shapely.geometry import Point
from pylandsat import Catalog, Product

catalog = Catalog()

begin = datetime(2010, 1, 1)
end = datetime(2020, 1, 1)
geom = Point(4.34, 50.85)

# Results are returned as a list
scenes = catalog.search(
    begin=begin,
    end=end,
    geom=geom,
    sensors=['LE07', 'LC08']
)

# Get the product ID of the first scene
product_id = scenes[0].get("product_id")

# Download the scene
product = Product(product_id)
product.download(out_dir='data')

# The output of catalog.search() can be converted to a DataFrame
# for further processing. For instance:
# Get the product ID of the scene with the lowest cloud cover
import pandas as pd

df = pd.DataFrame.from_dict(scenes)
df.set_index(["product_id"], inplace=True)
df = df.sort_values(by='cloud_cover', ascending=True)
product_id = df.index[0]

OperationalError: Le module spécifié est introuvable.


In [5]:
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from pylandsat import Scene

# Access data
scene = Scene('data/LE07_L1TP_205050_19991104_20170216_01_T1')
print(scene.available_bands())
print(scene.product_id)
print(scene.sensor)
print(scene.date)

# Access MTL metadata
print(scene.mtl['IMAGE_ATTRIBUTES']['CLOUD_COVER_LAND'])

# Quality band
plt.imshow(scene.quality.read())

# Access band data
nir = scene.nir.read(1)
red = scene.red.read(1)
ndvi = (nir + red) / (nir - red)

# Access band metadata
print(scene.nir.bname)
print(scene.nir.fname)
print(scene.nir.profile)
print(scene.nir.width, scene.nir.height)
print(scene.nir.crs)

# Use reflectance values instead of DN
nir = scene.nir.to_reflectance()

# ..or brightness temperature
tirs = scene.tirs.to_brightness_temperature()

# Save file to disk
with rasterio.open('temperature.tif', 'w', **scene.tirs.profile) as dst:
    dst.write(tirs, 1)

FileNotFoundError: [WinError 3] Le chemin d’accès spécifié est introuvable: 'c:\\Users\\marga\\Documents\\EPFL\\Cours\\S2\\Design_project\\codes\\Satellite_glacier\\draft\\data\\LE07_L1TP_205050_19991104_20170216_01_T1'

# Using Landsatxplore

In [6]:
pip install landsatxplore


   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 1.3/1.3 MB 7.4 MB/s eta 0:00:00
  Attempting uninstall: shapely
    Found existing installation: shapely 2.0.6
    Uninstalling shapely-2.0.6:
      Successfully uninstalled shapely-2.0.6
  Attempting uninstall: click
    Found existing installation: click 8.1.7
    Uninstalling click-8.1.7:
      Successfully uninstalled click-8.1.7
Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fiona 1.10.1 requires click~=8.0, but you have click 7.1.2 which is incompatible.
geopandas 1.0.1 requires shapely>=2.0.0, but you have shapely 1.8.5.post1 which is incompatible.
libpysal 4.12.1 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.


In [ ]:
from landsatxplore.api import API

# Initialize a new API instance and get an access key
api = API(username, password)

# Perform a request. Results are returned in a dictionnary
response = api.request(
    '<request_endpoint>',
    params={
        "param_1": value_1,
        "param_2": value_2
    }
)

# Log out
api.logout()

In [ ]:
import json
from landsatxplore.api import API

# Initialize a new API instance and get an access key
api = API(username, password)

# Search for Landsat TM scenes
scenes = api.search(
    dataset='landsat_tm_c2_l1',
    latitude=50.85,
    longitude=-4.35,
    start_date='1995-01-01',
    end_date='1995-10-01',
    max_cloud_cover=10
)

print(f"{len(scenes)} scenes found.")

# Process the result
for scene in scenes:
    print(scene['acquisition_date'].strftime('%Y-%m-%d'))
    # Write scene footprints to disk
    fname = f"{scene['landsat_product_id']}.geojson"
    with open(fname, "w") as f:
        json.dump(scene['spatial_coverage'].__geo_interface__, f)

api.logout()

In [ ]:
from landsatxplore.earthexplorer import EarthExplorer

ee = EarthExplorer(username, password)

ee.download('LT51960471995178MPS00', output_dir='./data')

ee.logout()